## FILTRO CON MASCARAS PARA FILTRAR SONIDO DE BATERIA

In [ ]:
from scipy.signal import butter, lfilter
from scipy.io import wavfile as waves
from IPython.display import Audio
import scipy.fftpack as fourier
import pyaudio as pa   
import numpy as np
import matplotlib.pyplot as plt

filename = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Data/Idles.wav'

Fs, data = waves.read(filename) # Leer el archivo y encontrar sus frecuencias y componentes

if len(data.shape) > 1: # Comprobar que hay datos
    Audio_m = data [:,0] # [componentes, canal]
else:
    Audio_m = data

# Tamaño de la señal
L = len(Audio_m)
print(L)
n = np.arange(0,L)/(Fs*0.25) # Teorema de muestreo

fft = fourier.fft(Audio_m)   # Realizar la FFT
M_fft = abs(fft)   # Magnitud de la señal
M_fft = M_fft[0:L//2]   # Mitad de los datos

F = Fs*np.arange(0,L//2)/L 
frecuencias = np.fft.fftfreq(L,1/Fs) #  Frecuencias en mi archivo de audio

fc_1 = 200
fc_2 = 800

# Creamos una sola máscara que cumpla ambas condiciones para filtro pasa-banda
filtro_mask = (np.abs(frecuencias) >= fc_1) & (np.abs(frecuencias) <= fc_2)
# aplicar el filtro
X_filtrado = fft * filtro_mask 

# retornamos señal al dominio del tiempo
audio_filtrado = np.fft.ifft(X_filtrado)
audio_filtrado = np.real(audio_filtrado)

# Guardar el nuevo archivo
ruta = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Tarea2/Guitarra.wav'
waves.write(ruta,Fs,audio_filtrado.astype(np.int16))

fft = fourier.fft(Audio_m)   # realizar la FFT
M_fft = abs(fft)   # Magnitud de la señal
M_fft = M_fft[0:L//2]   # Mitad de los datos

#  GRAFICAR
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Espectro original vs Filtrado
ax1.plot(frecuencias[:L//2], np.abs(fft[:L//2]), label="Original", alpha=0.5)
ax1.plot(frecuencias[:L//2], np.abs(X_filtrado[:L//2]), label="Filtrado (Pasa-Banda)", color='r')
ax1.set_title(f"Filtro Pasa-Banda en Frecuencia (Banda: {fc_1} - {fc_2} Hz) Señal Guitarra")
ax1.set_xlim(0, 4000)
ax1.legend()

# Señal en el tiempo
ax2.plot(n, Audio_m, label="Original", alpha=0.5)
ax2.plot(n, audio_filtrado, label="Filtrado", color='g')
ax2.set_title("Comparación en el Tiempo")
ax2.set_xlabel("Tiempo [s]")
ax2.legend()

plt.tight_layout()
plt.show()


## FILTRO IIR PARA FILTRAR SONIDO DE BATERIA

In [ ]:
from scipy.signal import butter, lfilter
from scipy.io import wavfile as waves
from IPython.display import Audio
import scipy.fftpack as fourier
import pyaudio as pa   
import numpy as np
import matplotlib.pyplot as plt

filename = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Tarea2/Idles.wav'

Fs, data = waves.read(filename) # Leer el archivo y encontrar sus frecuencias y componentes

if len(data.shape) > 1: # Comprobar que hay datos
    Audio_m = data [:,0] # [componentes, canal]
else:
    Audio_m = data

data_norm = data /np.max(np.abs(data))
L = len(data_norm)
fft = np.fft.fft(data_norm)
frecuencias = np.fft.fftfreq(L,1/Fs)

def butter_highpass(fc,fs,order): 
    nyq = 0.5 * fs
    normal_fc = fc / nyq
    b , a = butter(order, normal_fc, btype='high', analog=False)
    return b,a  


#### filtro pasa altos
fc = 10000
b,a = butter_highpass(fc,Fs,order= 2)
# aplicar el filtro
audio_filtrado_iir = lfilter (b,a,data_norm)

fft_filtrada = np.fft.fft(audio_filtrado_iir)
# escalar a 16 bits
audio_final = (audio_filtrado_iir*32767).astype(np.int16)
ruta = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Tarea2/guitarra_pa.wav'
waves.write(ruta,Fs,audio_final)

t = np.arange(L) / Fs

# 1. Configuración de la figura
plt.figure(figsize=(12, 10))
plt.subplots_adjust(hspace=0.4)

# --- DOMINIO DEL TIEMPO ---
plt.subplot(2, 1, 1)
plt.plot(t, data_norm, label='Original', color='lightgray', alpha=0.7)
plt.plot(t, audio_filtrado_iir, label='Filtrado (Pasa Altos)', color='blue', alpha=0.8)
plt.title('Señal en el Tiempo')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud Normalizada')
plt.legend()
plt.grid(True)

# --- DOMINIO DE LA FRECUENCIA ---
# Calculamos la magnitud de la FFT (solo la parte positiva)
mag_original = np.abs(fft)
mag_filtrada = np.abs(fft_filtrada)

# Solo nos interesa la mitad positiva del espectro
half_L = L // 2
f_plot = frecuencias[:half_L]
mag_orig_plot = mag_original[:half_L]
mag_filt_plot = mag_filtrada[:half_L]

plt.subplot(2, 1, 2)
plt.plot(f_plot, mag_orig_plot, label='Espectro Original', color='lightgray')
plt.plot(f_plot, mag_filt_plot, label='Espectro Filtrado', color='red')
plt.axvline(fc, color='green', linestyle='--', label=f'Corte: {fc} Hz') # Línea de corte
plt.title('Análisis de Frecuencia (FFT)')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud')
plt.xlim(0, Fs/2) # Límite hasta la frecuencia de Nyquist
plt.legend()
plt.grid(True)

plt.show()


## FILTRADO DE GUITARRA ELECTRICA CON FILTRO DE MASCARAS

In [ ]:
from scipy.signal import butter, lfilter
from scipy.io import wavfile as waves
from IPython.display import Audio
import scipy.fftpack as fourier
import pyaudio as pa   
import numpy as np
import matplotlib.pyplot as plt

filename = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Tarea2/Idles.wav'

Fs, data = waves.read(filename) # Leer el archivo y encontrar sus frecuencias y componentes

if len(data.shape) > 1: # Comprobar que hay datos
    Audio_m = data [:,0] # [componentes, canal]
else:
    Audio_m = data

# Tamaño de la señal
L = len(Audio_m)
print(L)
n = np.arange(0,L)/(Fs*0.25) # Teorema de muestreo

fft = fourier.fft(Audio_m)   # Realizar la FFT
M_fft = abs(fft)   # Magnitud de la señal
M_fft = M_fft[0:L//2]   # Mitad de los datos

F = Fs*np.arange(0,L//2)/L 
frecuencias = np.fft.fftfreq(L,1/Fs) #  Frecuencias en mi archivo de audio
fc_1 = 100
fc_2 = 8000

# Creamos una sola máscara que cumpla ambas condiciones para filtro rechaza-banda
filtro_mask = (np.abs(frecuencias) <= fc_1) | (np.abs(frecuencias) >= fc_2)
# aplicar el filtro
X_filtrado = fft * filtro_mask 

# retornamos señal al dominio del tiempo
audio_filtrado = np.fft.ifft(X_filtrado)
audio_filtrado = np.real(audio_filtrado)

# Guardar el nuevo archivo
ruta = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Tarea2/Batería.wav'
waves.write(ruta,Fs,audio_filtrado.astype(np.int16))

fft = fourier.fft(Audio_m)   # realizar la FFT
M_fft = abs(fft)   # Magnitud de la señal
M_fft = M_fft[0:L//2]   # Mitad de los datos

#  GRAFICAR
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Espectro original vs Filtrado
ax1.plot(frecuencias[:L//2], np.abs(fft[:L//2]), label="Original", alpha=0.5)
ax1.plot(frecuencias[:L//2], np.abs(X_filtrado[:L//2]), label="Filtrado (Rechaza-Banda)", color='r')
ax1.set_title(f"Filtro Rechaza-Banda en Frecuencia (Banda: {fc_1} - {fc_2} Hz) Señal Batería")
ax1.set_xlim(0, 10000)
ax1.legend()

# Señal en el tiempo
ax2.plot(n, Audio_m, label="Original", alpha=0.5)
ax2.plot(n, audio_filtrado, label="Filtrado", color='g')
ax2.set_title("Comparación en el Tiempo")
ax2.set_xlabel("Tiempo [s]")
ax2.legend()

plt.tight_layout()
plt.show()



## FILTRADO DE GUITARRA ELECTRICA CON FILTRO IIR

In [ ]:
from scipy.signal import butter, lfilter
from scipy.io import wavfile as waves
from IPython.display import Audio
import scipy.fftpack as fourier
import pyaudio as pa   
import numpy as np
import matplotlib.pyplot as plt

filename = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Tarea2/Idles.wav'

Fs, data = waves.read(filename) # Leer el archivo y encontrar sus frecuencias y componentes

if len(data.shape) > 1: # Comprobar que hay datos
    Audio_m = data [:,0] # [componentes, canal]
else:
    Audio_m = data

data_norm = data /np.max(np.abs(data))
L = len(data_norm)
fft = np.fft.fft(data_norm)
frecuencias = np.fft.fftfreq(L,1/Fs)

#### filtro rechaza banda    
def butter_bandstop(fc_low, fc_high, fs, order=4):
    nyq = 0.5 * fs
    low = fc_low / nyq
    high = fc_high / nyq
    # El parámetro btype='bandstop' es la clave aquí
    b, a = butter(order, [low, high], btype='bandstop', analog=False)
    return b, a

#### Ejecucion del filtro rechaza banda
fc_1 = 200.0
fc_2 = 8000.0
b,a = butter_bandstop(fc_1,fc_2,Fs,order= 2)
# aplicar el filtro
audio_filtrado_iir = lfilter (b,a,data_norm)

fft_filtrada = np.fft.fft(audio_filtrado_iir)
# escalar a 16 bits
audio_final = (audio_filtrado_iir*32767).astype(np.int16)
ruta = 'C:/Users/santi/Maestria/Procesamiento de Imagenes/Clase 2/Tarea2/Bateria_rb_200_2kHz.wav'
waves.write(ruta,Fs,audio_final)

# 1. Configuración de la figura
plt.figure(figsize=(12, 10))
plt.subplots_adjust(hspace=0.4)

# --- DOMINIO DEL TIEMPO ---
t = np.arange(L) / Fs

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Espectro original vs Filtrado
ax1.plot(frecuencias[:L//2], np.abs(fft[:L//2]), label="Original", alpha=0.5)
ax1.plot(frecuencias[:L//2], np.abs(fft_filtrada[:L//2]), label="Filtrado (Pasa-Banda)", color='r')
ax1.set_xlim(0, 10000) # Zoom a la zona de interés
ax1.set_ylabel("Magnitud")
ax1.legend()

# Señal en el tiempo
ax2.plot(t, data_norm, label="Original", alpha=0.5)
ax2.plot(t, audio_filtrado_iir, label="Filtrado", color='g')
ax2.set_title("Comparación en el Tiempo")
ax2.set_xlabel("Tiempo [s]")
ax2.set_ylabel("Amplitud")
ax2.legend()
